In [1]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

df_1_3 = pd.read_csv('main_table.tsv',  sep='\t')
df_aug = pd.read_csv('6_augmentation.tsv',  sep='\t')
df_4_bgpd_gbm = pd.read_csv('transfer_bgpd_gbm.tsv',  sep='\t')
df_4_gbm_bgpd = pd.read_csv('transfer_gbm_bgpd.tsv',  sep='\t')

df_1_3.replace([np.inf, -np.inf], np.nan, inplace=True)
df_aug.replace([np.inf, -np.inf], np.nan, inplace=True)
df_4_bgpd_gbm.replace([np.inf, -np.inf], np.nan, inplace=True)
df_4_gbm_bgpd.replace([np.inf, -np.inf], np.nan, inplace=True)

In [2]:
df_1_3 = df_1_3[df_1_3['Hausdorff'].notna()]
df_aug = df_aug[df_aug['Hausdorff'].notna()]
df_4_bgpd_gbm = df_4_bgpd_gbm[df_4_bgpd_gbm['Hausdorff'].notna()]
df_4_gbm_bgpd = df_4_gbm_bgpd[df_4_gbm_bgpd['Hausdorff'].notna()]

In [3]:
df_1_3.head()

,PatientID,Preprocessing step,Dataset,"NN-Architecture (nn-Unet, UNETR)","Label (WT, ET, TC, GTV)",Dice,Hausdorff,Surface Dice 1mm,Surface Dice 3mm,Surface Dice 5mm,Epoch,Volume_estimated,Volume_intersection,Volume_true
0,TCGA-02-0011,1_reg,gbm,nn_unet,WT,0.290,39.000,0.264,NaN,NaN,300,25.748,22.114,126.783
1,TCGA-02-0011,1_reg,gbm,nn_unet,ET,0.138,39.000,0.218,NaN,NaN,300,2.983,22.114,33.488
2,TCGA-02-0011,1_reg,gbm,nn_unet,TC,0.030,34.300,0.133,NaN,NaN,300,9.525,22.114,81.619
3,TCGA-02-0033,1_reg,gbm,nn_unet,WT,0.242,39.000,0.261,NaN,NaN,300,19.781,14.996,104.407
4,TCGA-02-0033,1_reg,gbm,nn_unet,ET,0.265,26.094,0.330,NaN,NaN,300,6.324,14.996,31.703


In [4]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [5]:


reindex_list = ['1_reg', '2a_interp', '3a_atlas', '4a_resamp', '4b_n4', '4d_susan','6_hist','5_ss_shared']
rename_list = ['1. Inter modality', '2. Resampling', '3. Atlas', '4. Resampling to spacing', '4.a Bias', '4.b Denoising', '4.c Histogram', '4.d Skull']

dfs_list = []
dataframes = df_1_3, df_aug, df_4_bgpd_gbm, df_4_gbm_bgpd

In [6]:
for df in dataframes:
    dfs_list.append(get_df_name(df))
#dfs_list[2] = 'df_4_bgpd_gbm'
dfs_list

['df_1_3', 'df_aug', 'df_4_bgpd_gbm', 'df_4_gbm_bgpd']

In [7]:
df_4_bgpd_gbm["Preprocessing step"].unique()

array(['1_reg', '2a_interp', '3a_atlas', '4a_resamp', '4b_n4', '4d_susan',
       '5_ss_shared', '6_hist'], dtype=object)

In [8]:
print(dfs_list)

['df_1_3', 'df_aug', 'df_4_bgpd_gbm', 'df_4_gbm_bgpd']


In [9]:
dfs_and_steps = {
    dfs_list[0]: ['4. Resampling to spacing', '4.d Skull'],
    dfs_list[1]: ['4. Resampling to spacing', '4.d Skull'],
    dfs_list[2]: ['4. Resampling to spacing', '4.d Skull'],
    dfs_list[3]: ['4. Resampling to spacing', '4.d Skull']
}


In [10]:
dfs_and_epoches = {
    dfs_list[0]: (100, 300),
    dfs_list[1]: 300,
    dfs_list[2]: (100, 300),
    dfs_list[3]: (100, 300)
}

In [11]:
dfs_and_steps[get_df_name(df_1_3)]

['4. Resampling to spacing', '4.d Skull']

In [12]:
#Table 4 DICE WT
#models = df_1_3['NN-Architecture (nn-Unet, UNETR)'].unique()
epochs = (100, 300)
metrics = ['Dice', 'Hausdorff', 'Surface Dice 1mm']
datas = ['gbm', 'bgpd']
metrics_and_multilpiers = {
    metrics[0]: 100,
    metrics[1]: 1,
    metrics[2]: 100,
}

    
#metric = 'Dice'
#data = 'gbm'
label = 'WT'
model = 'nn_unet'
#epoch = 300
# to save space in the paper and not write 0.44 and write 44 instead (only for Dice and Surface Dice)
#multiplier = 100


def get_column_from_pivot_table(df, metric, data, label, model, epoch, multiplier):
    
    # mean
    temp_series = (np.round(df[df.Dataset==data][df.Epoch==epoch][df['Label (WT, ET, TC, GTV)']==label][df['NN-Architecture (nn-Unet, UNETR)']==model ].groupby([ 'Preprocessing step'])[metric].mean(),3)*multiplier).round(1)
    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_mean = temp_series
        ###############################################
# Apply the condition using boolean indexing
    temp_series_mean = temp_series_mean[dfs_and_steps[get_df_name(df)]]
    ###################################################

    #std
    temp_series = (np.round(df[df.Dataset==data][df.Epoch==epoch][df['Label (WT, ET, TC, GTV)']==label][df['NN-Architecture (nn-Unet, UNETR)']==model].groupby([ 'Preprocessing step'])[metric].std(),3)*multiplier).round(1)

    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_std = temp_series
    ###############################################
# Apply the condition using boolean indexing
    temp_series_std = temp_series_std[dfs_and_steps[get_df_name(df)]]
    ###################################################
    temp_mean_std = pd.concat([temp_series_mean, temp_series_std], axis=1)
    temp_mean_std.columns = ['Mean', 'Std']
    temp_mean_std['Mean (STD)'] = temp_mean_std.apply(lambda x: str(x['Mean']) + ' (' + str(x['Std']) + ')', axis=1)
    return temp_mean_std['Mean (STD)'] 

print('NEW TABLE FROM GOOGLE SHEETS')

for df in dataframes:
    print('==========================================', 'NEW DATAFRAME', get_df_name(df), '=========================================='),

    for epoch in epochs:
        print('============================', 'NEW #EPOCHES', '============================'),
        for data in datas:
            print('=====================', 'NEW DATA', '====================='),
            for metric in metrics:
                print(data, model, metric, epoch, ' epoches')
                print(get_column_from_pivot_table(df, metric, data, label, model, epoch, metrics_and_multilpiers[metric]))
                print( )

NEW TABLE FROM GOOGLE SHEETS
========================================== NEW DATAFRAME df_1_3 ==========================================
============================ NEW #EPOCHES ============================
===================== NEW DATA =====================
gbm nn_unet Dice 100  epoches
4. Resampling to spacing    83.7 (12.7)
4.d Skull                   86.0 (10.5)
Name: Mean (STD), dtype: object

gbm nn_unet Hausdorff 100  epoches
4. Resampling to spacing    15.3 (20.4)
4.d Skull                   10.6 (12.8)
Name: Mean (STD), dtype: object

gbm nn_unet Surface Dice 1mm 100  epoches
4. Resampling to spacing    62.2 (21.8)
4.d Skull                   66.6 (20.1)
Name: Mean (STD), dtype: object

===================== NEW DATA =====================
bgpd nn_unet Dice 100  epoches
4. Resampling to spacing    72.7 (16.5)
4.d Skull                   75.7 (14.6)
Name: Mean (STD), dtype: object

bgpd nn_unet Hausdorff 100  epoches
4. Resampling to spacing    35.3 (47.7)
4.d Skull            

По каждой метрике проверяем на gbm, bgpd, lgg. Делаем для nnU-net и unetr.

In [13]:
#Table 4 DICE WT
models = df_1_3['NN-Architecture (nn-Unet, UNETR)'].unique()
metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300
# to save space in the paper and not write 0.44 and write 44 instead (only for Dice and Surface Dice)
multiplier = 100


def get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier):
    
    # mean
    temp_series = (np.round(df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model ].groupby([ 'Preprocessing step'])[metric].mean(),3)*multiplier).round(1)
    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_mean = temp_series

    #std
    temp_series = (np.round(df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model].groupby([ 'Preprocessing step'])[metric].std(),3)*multiplier).round(1)

    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_std = temp_series

    temp_mean_std = pd.concat([temp_series_mean, temp_series_std], axis=1)
    temp_mean_std.columns = ['Mean', 'Std']
    temp_mean_std['Mean (STD)'] = temp_mean_std.apply(lambda x: str(x['Mean']) + ' (' + str(x['Std']) + ')', axis=1)
    return temp_mean_std['Mean (STD)'] 

print('DICE scores')
for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print(data, model, metric)
        print(get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier))
        print( )

DICE scores
=======================NEW MODEL=======================
gbm nn_unet Dice
1. Inter modality           43.6 (27.7)
2. Resampling               84.8 (11.4)
3. Atlas                    84.6 (11.3)
4. Resampling to spacing    85.0 (11.6)
4.a Bias                    82.4 (13.1)
4.b Denoising               84.5 (11.6)
4.c Histogram               83.3 (16.1)
4.d Skull                   86.6 (10.7)
Name: Mean (STD), dtype: object

bgpd nn_unet Dice
1. Inter modality           37.2 (28.8)
2. Resampling               72.7 (18.6)
3. Atlas                    75.3 (15.8)
4. Resampling to spacing    73.8 (17.6)
4.a Bias                    75.9 (15.4)
4.b Denoising               74.5 (16.5)
4.c Histogram               75.4 (14.9)
4.d Skull                   76.1 (14.4)
Name: Mean (STD), dtype: object

lgg nn_unet Dice
1. Inter modality           67.0 (26.9)
2. Resampling               72.1 (23.9)
3. Atlas                    70.8 (25.0)
4. Resampling to spacing    70.4 (24.8)
4.a Bias      

In [14]:
# Appending Table  Hausdorff WT
metric = 'Hausdorff'
data = 'gbm'
label = 'WT'
models = df_1_3['NN-Architecture (nn-Unet, UNETR)'].unique()

#get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=1)

print('Hausdorff scores')
for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print(data, model, metric)
        print(get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=1))
        print( )

Hausdorff scores
=======================NEW MODEL=======================
gbm nn_unet Hausdorff
1. Inter modality           30.6 (21.0)
2. Resampling               15.4 (20.6)
3. Atlas                    10.0 (11.1)
4. Resampling to spacing    13.1 (19.7)
4.a Bias                    14.1 (22.3)
4.b Denoising               11.5 (15.6)
4.c Histogram               13.2 (18.1)
4.d Skull                   10.3 (13.6)
Name: Mean (STD), dtype: object

bgpd nn_unet Hausdorff
1. Inter modality           45.5 (38.6)
2. Resampling               26.0 (33.4)
3. Atlas                    21.5 (27.1)
4. Resampling to spacing    23.5 (34.1)
4.a Bias                    21.6 (34.4)
4.b Denoising               25.6 (39.1)
4.c Histogram               21.9 (34.1)
4.d Skull                   15.3 (17.3)
Name: Mean (STD), dtype: object

lgg nn_unet Hausdorff
1. Inter modality           35.1 (36.3)
2. Resampling               29.5 (32.2)
3. Atlas                    26.7 (34.8)
4. Resampling to spacing    38.0 (

In [15]:
# Appending Table  Surface Dice WT
metric = 'Surface Dice 1mm'
data = 'gbm'
label = 'WT'
model = 'nn_unet'

print('Surface Dice WT')
for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print(data, model, metric)
        print(get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=100))
        print( )

Surface Dice WT
=======================NEW MODEL=======================
gbm nn_unet Surface Dice 1mm
1. Inter modality           26.7 (14.9)
2. Resampling               64.9 (20.0)
3. Atlas                    63.2 (20.0)
4. Resampling to spacing    65.4 (20.5)
4.a Bias                    60.2 (21.9)
4.b Denoising               63.6 (20.9)
4.c Histogram               63.0 (21.8)
4.d Skull                   68.6 (20.3)
Name: Mean (STD), dtype: object

bgpd nn_unet Surface Dice 1mm
1. Inter modality           21.0 (14.0)
2. Resampling               40.8 (18.0)
3. Atlas                    43.7 (17.3)
4. Resampling to spacing    42.7 (18.3)
4.a Bias                    44.6 (18.2)
4.b Denoising               42.7 (17.6)
4.c Histogram               43.5 (17.6)
4.d Skull                   45.3 (18.6)
Name: Mean (STD), dtype: object

lgg nn_unet Surface Dice 1mm
1. Inter modality           41.4 (24.4)
2. Resampling               47.2 (23.8)
3. Atlas                    45.6 (25.5)
4. Resampling 

In [16]:
## How to check where to put rows on the statistical significance:

def dropna_intersection(data_1: pd.Series, data_2: pd.Series):
    data_1 = data_1.reset_index(drop=True)
    data_2 = data_2.reset_index(drop=True)

    data_1 = data_1.dropna()
    data_2 = data_2.dropna()

    data_1 = data_1[data_1.index.isin(data_2.index)]
    data_2 = data_2[data_2.index.isin(data_1.index)]
    
    return data_1, data_2

In [18]:
import scipy
from statsmodels.stats.weightstats import ttost_ind, ttost_paired

metric = 'Hausdorff'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300


preproc_1 = "4a_resamp"
for key in reindex_list:
    preproc_2 = key
    
    if key != "4a_resamp":

        #data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
        #data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
        data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
        data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]

        
        ttost = ttost_paired(np.array(data_1), np.array(data_2), -data_2.std()/4, data_2.std()/4)[0]/7
        ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

        # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
        if ttost > 0.05:
            print(preproc_2, (np.round(data_1.mean(),3)*1).round(0).astype(int), '(',
              (np.round(data_1.std(),3)*1).round(0).astype(int), ')',
              ' vs ',\
              (np.round(data_2.mean(),3)*1).round(0).astype(int), '(' ,
              (np.round(data_2.std(),3)*1).round(0).astype(int),')')
            print( "Difference is significant, p-value ", np.round(ttest_,3))

1_reg 13 ( 20 )  vs  31 ( 21 )
Difference is significant, p-value  0.0
3a_atlas 13 ( 20 )  vs  10 ( 11 )
Difference is significant, p-value  6.359
5_ss_shared 13 ( 20 )  vs  10 ( 14 )
Difference is significant, p-value  0.104


# 10.02.2024*** BEGIN

In [19]:
metrics_and_multilpiers

{'Dice': 100, 'Hausdorff': 1, 'Surface Dice 1mm': 100}

In [20]:
shorten_reindex_list=['4a_resamp', '5_ss_shared']
not_keys = [
'5_ss_shared',
'4a_resamp'
]

In [22]:
#metric = 'Dice'
#data = 'gbm'
label = 'WT'
model = 'nn_unet'
#epoch = 300

for df in dataframes:
    print('=======================NEW DF', get_df_name(df), '=======================')
    for metric in metrics:
        for data in datas:
            preproc_1 = "4a_resamp"
            for key in shorten_reindex_list:
                preproc_2 = key
                if key == "4a_resamp":

                    data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==300][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                    data_2 = df[df.Dataset==data][df.Epoch==100][df['Label (WT, ET, TC, GTV)']==label][df['NN-Architecture (nn-Unet, UNETR)']==model][df['Preprocessing step']==preproc_2][metric]
                    data_1, data_2 = dropna_intersection(data_1, data_2)

                    ttost = ttost_paired(np.array(data_1), np.array(data_2), -data_2.std()/4, data_2.std()/4)[0]/7
                    ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

                    #assert isna
                    assert data_1.isnull().sum() == 0
                    assert data_2.isnull().sum() == 0

                    # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                    if ttest_ > 0.05:
                        print(data, model, metric, epoch, 'epoches')
                        print(preproc_2, (np.round(data_1.mean(),3)*metrics_and_multilpiers[metric]).round(0).astype(int), '(',
                          (np.round(data_1.std(),3)*metrics_and_multilpiers[metric]).round(0).astype(int), ')',
                          ' vs ',\
                          (np.round(data_2.mean(),3)*metrics_and_multilpiers[metric]).round(0).astype(int), '(' ,
                          (np.round(data_2.std(),3)*metrics_and_multilpiers[metric]).round(0).astype(int),')')
                        print( "Difference is significant, p-value ", np.round(ttest_,3))
                        print( )

=======================NEW DF df_1_3 =======================
bgpd nn_unet Dice 300 epoches
4a_resamp 74 ( 18 )  vs  73 ( 16 )
Difference is significant, p-value  0.279

bgpd nn_unet Hausdorff 300 epoches
4a_resamp 23 ( 34 )  vs  35 ( 48 )
Difference is significant, p-value  0.073

=======================NEW DF df_aug =======================
=======================NEW DF df_4_bgpd_gbm =======================
gbm nn_unet Dice 300 epoches
4a_resamp 85 ( 12 )  vs  86 ( 10 )
Difference is significant, p-value  5.033

gbm nn_unet Hausdorff 300 epoches
4a_resamp 13 ( 20 )  vs  11 ( 15 )
Difference is significant, p-value  3.823

gbm nn_unet Surface Dice 1mm 300 epoches
4a_resamp 65 ( 20 )  vs  65 ( 20 )
Difference is significant, p-value  1.321

=======================NEW DF df_4_gbm_bgpd =======================
bgpd nn_unet Dice 300 epoches
4a_resamp 74 ( 18 )  vs  72 ( 17 )
Difference is significant, p-value  0.42

bgpd nn_unet Hausdorff 300 epoches
4a_resamp 23 ( 34 )  vs  31 ( 46 )
Differ

In [406]:
preproc_1 = "4a_resamp"
preproc_2 = "4a_resamp"


data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==300][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
data_2 = df_aug[df_aug.Dataset==data][df_aug.Epoch==300][df_aug['Label (WT, ET, TC, GTV)']==label][df_aug['NN-Architecture (nn-Unet, UNETR)']==model][df_aug['Preprocessing step']==preproc_2][metric]
data_1, data_2 = dropna_intersection(data_1, data_2)

ttost = ttost_paired(np.array(data_1), np.array(data_2), -data_2.std()/4, data_2.std()/4)[0]/7
ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

#assert isna
assert data_1.isnull().sum() == 0
assert data_2.isnull().sum() == 0

# print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
if ttest_ > 0.05:
    print(data, model, metric, epoch, 'epoches')
    print(preproc_2, (np.round(data_1.mean(),3)*metrics_and_multilpiers[metric]).round(0).astype(int), '(',
      (np.round(data_1.std(),3)*metrics_and_multilpiers[metric]).round(0).astype(int), ')',
      ' vs ',\
      (np.round(data_2.mean(),3)*metrics_and_multilpiers[metric]).round(0).astype(int), '(' ,
      (np.round(data_2.std(),3)*metrics_and_multilpiers[metric]).round(0).astype(int),')')
    print( "Difference is significant, p-value ", np.round(ttest_,3))
    print( )

bgpd nn_unet Surface Dice 1mm 300 epoches
4a_resamp 42 ( 18 )  vs  43 ( 19 )
Difference is significant, p-value  5.468



# 10.02.2024*** END

In [72]:
metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        preproc_1 = "4a_resamp"
        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
                data_1, data_2 = dropna_intersection(data_1, data_2)

                ttost = ttost_paired(np.array(data_1), np.array(data_2), -data_2.std()/4, data_2.std()/4)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7
                                
                #assert isna
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if ttost > 0.05:
                    print(data, model, metric)
                    print(preproc_2, (np.round(data_1.mean(),3)*100).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*100).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*100).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*100).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttest_,3))
                    print( )

=======================NEW MODEL=======================
gbm nn_unet Dice
1_reg 85 ( 12 )  vs  44 ( 28 )
Difference is significant, p-value  0.0

bgpd nn_unet Dice
1_reg 74 ( 18 )  vs  37 ( 29 )
Difference is significant, p-value  0.0

lgg nn_unet Dice
5_ss_shared 70 ( 25 )  vs  77 ( 21 )
Difference is significant, p-value  0.0

=======================NEW MODEL=======================
gbm UNETR Dice
1_reg 82 ( 14 )  vs  39 ( 26 )
Difference is significant, p-value  0.0

gbm UNETR Dice
5_ss_shared 82 ( 14 )  vs  85 ( 10 )
Difference is significant, p-value  0.158

bgpd UNETR Dice
1_reg 68 ( 21 )  vs  35 ( 30 )
Difference is significant, p-value  0.0

bgpd UNETR Dice
4b_n4 68 ( 21 )  vs  73 ( 18 )
Difference is significant, p-value  0.0

bgpd UNETR Dice
5_ss_shared 68 ( 21 )  vs  72 ( 18 )
Difference is significant, p-value  0.0

lgg UNETR Dice
4b_n4 67 ( 23 )  vs  62 ( 22 )
Difference is significant, p-value  0.159

lgg UNETR Dice
5_ss_shared 67 ( 23 )  vs  75 ( 19 )
Difference is signifi

In [55]:
metric = 'Hausdorff'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        preproc_1 = "4a_resamp"

        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
 
                data_1, data_2 = dropna_intersection(data_1, data_2)
                
                #assert isna
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0
                
                ttost = ttost_paired(np.array(data_1), np.array(data_2), -data_2.std()/4, data_2.std()/4)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if  (ttost > 0.05):
                    print(data, model, metric)
                    print(preproc_2, (np.round(data_1.mean(),3)*1).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*1).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*1).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*1).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttest_,3), np.round(ttost,3))
                    print( )

=======================NEW MODEL=======================
gbm nn_unet Hausdorff
1_reg 13 ( 20 )  vs  31 ( 21 )
Difference is significant, p-value  0.0 0.143

gbm nn_unet Hausdorff
3a_atlas 13 ( 20 )  vs  10 ( 11 )
Difference is significant, p-value  6.359 0.08

gbm nn_unet Hausdorff
5_ss_shared 13 ( 20 )  vs  10 ( 14 )
Difference is significant, p-value  0.104 0.055

bgpd nn_unet Hausdorff
1_reg 23 ( 34 )  vs  46 ( 39 )
Difference is significant, p-value  0.0 0.143

bgpd nn_unet Hausdorff
5_ss_shared 23 ( 34 )  vs  15 ( 17 )
Difference is significant, p-value  0.025 0.132

lgg nn_unet Hausdorff
2a_interp 38 ( 39 )  vs  29 ( 32 )
Difference is significant, p-value  0.037 0.084

lgg nn_unet Hausdorff
3a_atlas 38 ( 39 )  vs  27 ( 35 )
Difference is significant, p-value  0.213 0.109

lgg nn_unet Hausdorff
5_ss_shared 38 ( 39 )  vs  23 ( 25 )
Difference is significant, p-value  0.001 0.142

=======================NEW MODEL=======================
gbm UNETR Hausdorff
1_reg 16 ( 23 )  vs  49 ( 2

In [56]:
metric = 'Surface Dice 1mm'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        preproc_1 = "4a_resamp"
        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
                data_1, data_2 = dropna_intersection(data_1, data_2)
                
                #assert isna
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0

                ttost = ttost_ind(data_1, data_2, -data_2.std()/4, data_2.std()/4)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if ttost > 0.05:
                    print(data, model, metric)
                    print(preproc_2, (np.round(data_1.mean(),3)*100).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*100).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*100).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*100).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttest_,3))
                    print( )

=======================NEW MODEL=======================
gbm nn_unet Surface Dice 1mm
1_reg 65 ( 20 )  vs  27 ( 15 )
Difference is significant, p-value  0.0

gbm nn_unet Surface Dice 1mm
4b_n4 65 ( 20 )  vs  60 ( 22 )
Difference is significant, p-value  0.0

bgpd nn_unet Surface Dice 1mm
1_reg 42 ( 18 )  vs  21 ( 14 )
Difference is significant, p-value  0.0

lgg nn_unet Surface Dice 1mm
1_reg 46 ( 25 )  vs  41 ( 24 )
Difference is significant, p-value  0.0

lgg nn_unet Surface Dice 1mm
6_hist 46 ( 25 )  vs  42 ( 24 )
Difference is significant, p-value  0.157

lgg nn_unet Surface Dice 1mm
5_ss_shared 46 ( 25 )  vs  52 ( 23 )
Difference is significant, p-value  0.004

=======================NEW MODEL=======================
gbm UNETR Surface Dice 1mm
1_reg 59 ( 21 )  vs  22 ( 12 )
Difference is significant, p-value  0.0

gbm UNETR Surface Dice 1mm
4b_n4 59 ( 21 )  vs  52 ( 21 )
Difference is significant, p-value  0.0

gbm UNETR Surface Dice 1mm
5_ss_shared 59 ( 21 )  vs  64 ( 20 )
Differen